经过前面的铺垫，本节我们正式进入循环神经网络RNN，RNN也是首个针对于序列模型的神经网络
* 本节主要是概念性的东西，没什么代码

实际上RNN的假设本质上是一个MLP，这个用语言有点难以形容，具体的变量、数据变换关系可以参考李沐老师的视频
* 一个本质区别是，在MLP里面，隐藏层的feature也就是h，是只跟输入的x有关系的，比如说全连接层里面的一个h1，他就是前一层的全部feature通过某一权重生成的
* RNN里面同样是这样，但是除此之外，某一个隐藏层之间的h1、h2等等之间也是有关系的，因为这里存在时间关系，h2肯定会受到h1的影响
* 别忘了序列模型自始自终是一个自回归模型，我们是要通过此前时间的数据来预测当前时间、或者是往后时间的结果
* 语言模型实际上可以看做是一个分类问题（你每次预测下一个词，实际上都类似于做一个分类，从你的总语料库里面去分类），所以语言模型的损失可以看作一个交叉熵损失（预测n个词的话，那就是n个交叉熵损失的平均，一句话的损失就是这样）
* 由于历史原因，NLP的损失通常用exp(交叉熵)叫做困惑度来表示，困惑度为1就是最完美的情况，无穷大就是纯垃圾

在前面梯度爆炸\数值稳定性那一节我们有讲过，因为梯度更新的时候需要进行很多很多次矩阵乘法，所以梯度有时候会变得特别大或者特别小
* 在RNN里面，由于我们算梯度的时候需要考虑到时间的关系（比如T步时间点上，我们其实每次都是要更新梯度的），所以RNN相对来说更加容易出现梯度爆炸
* 这里我们会使用一种叫梯度剪裁的方法来控制梯度
* 梯度裁剪其实是，我们把整个网络里面的所有参数的梯度拿出来，组成一个向量，算个大小（L2范数）
* 如果这个大小超过了一定值，那我就按照这个设定好的threshold给他标准化了
* 这样就能有效防止梯度的爆炸（不过好像对梯度消失没啥作用？）
* 结合下面的代码能更好的理解

In [2]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

李沐老师的视频里面其实也提到了，比如说一个序列长度为T的RNN，其实你就可以把它看做一个特殊的，横过来的，层数为T的MLP，仔细想一想确实是这样的